**Video Processing & Video Object Tracking**

In [ ]:
!wget -O video.mp4 https://github.com/heshameraqi/Image_Processing_Computer_Vision_1/raw/main/tracking_slow_traffic_small.mp4

import numpy as np
import cv2
import matplotlib.pyplot as plt
#from IPython.display import clear_output # To clear old figure
#from time import sleep

# Capture video and take first frame of the video
cap = cv2.VideoCapture('video.mp4')
ret, frame = cap.read()

# Initial location of window r,h,c,w for tracked object
r,h,c,w = 200,20,300,20  
track_window = (c,r,w,h)

# Set up the object (ROI) for tracking
obj = frame[r:r+h, c:c+w]
hsv_obj =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # In OpenCV: H: 0-179, S: 0-255, V: 0-255
mask = cv2.inRange(hsv_obj, (0, 60,32), (180,255,255)) # Mask the vibrant colors (not white not black)
obj_hist = cv2.calcHist([hsv_obj],[0],mask,[180],[0,180]) # [0] means histogram of hsv_obj H channel only, masked by mask (vibrant), [180] bin count, in range of [0,180]

# Mean Shift algorithm termination criteria, either 10 iteration or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
i = 0
while(True):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # New frame
        # Back Projection: find a feature in the new frame image by seaching for its histogram model, returns probability image
        dst = cv2.calcBackProject([hsv],[0], obj_hist, [0,180], 1)

        # Apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw tracking result on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv2.imwrite("Result%04d.jpg" % i, img2)
        i+=1
        '''clear_output()
        plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        plt.show()
        sleep(1)'''
    else:
        break

cap.release() # Release video reader

# Create the resulted video file, then download it manually if needed and watch the result
!ffmpeg -framerate 30 -r 30 -i 'Result%04d.jpg' -vcodec mpeg4 -y result.mp4 # Save video
from google.colab import files
files.download('result.mp4')